# Import necessary packages

In [48]:
import copy
import sympy as sp
from sympy.matrices import ones, eye

# Input Variables

coords_original - Coordinates considered for distance measure in the hubo<br>torsional_bonds - Torsional bond numbers and their respective coordinate ends<br>coords_rotation_dict - Coordinate numbers and the bonds that affect them<br>distance_pairs - Pairs of coordinates to find distance

In [49]:
coords_original = {
    0: [1.0, -0.5, 0.0],
    1: [0.0, 0.0, 0.0],
    2: [0.0, 1.0, 0.0],
    3: [1.0, 1.5, 0.0],
    4: [2.0, 1.0, 0.0]
}
torsional_bonds = {
    0: (1, 2),
    1: (2, 3)
}
coords_rotation_dict = {
    0: [],
    1: [],
    2: [],
    3: [0],
    4: [0, 1]
}
distance_pairs = [
    (0, 3),
    (0, 4),
    (1, 3),
    (1, 4),
    (2, 4)]

In [50]:
coords_dict = copy.deepcopy(coords_original)
final_coords = copy.deepcopy(coords_original)

n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 4  # no. of discrete angles

x = []  # hubo variables

#Function to generate hard constraint

In [51]:
def generate_hard_constraint(include_a=False, a_value=1.0):
    for i in range(n_bonds):
        x.append(sp.symbols(f'x_{str(i)}_(0:{n_angles})'))
    hard_constraint = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[i][j]
        hard_constraint += (summation - 1) ** 2
    if include_a:
        a_const = sp.Symbol('A_const')
        hard_constraint *= a_const
    else:
        hard_constraint *= a_value
    return hard_constraint.expand()

In [52]:
hubo_expr = generate_hard_constraint(include_a=True)
sp.pprint(hubo_expr)

            2                                                                 
A_const⋅x₀ ₀  + 2⋅A_const⋅x₀ ₀⋅x₀ ₁ + 2⋅A_const⋅x₀ ₀⋅x₀ ₂ + 2⋅A_const⋅x₀ ₀⋅x₀ 

                                 2                                            
₃ - 2⋅A_const⋅x₀ ₀ + A_const⋅x₀ ₁  + 2⋅A_const⋅x₀ ₁⋅x₀ ₂ + 2⋅A_const⋅x₀ ₁⋅x₀ ₃

                                2                                             
 - 2⋅A_const⋅x₀ ₁ + A_const⋅x₀ ₂  + 2⋅A_const⋅x₀ ₂⋅x₀ ₃ - 2⋅A_const⋅x₀ ₂ + A_c

         2                                2                                   
onst⋅x₀ ₃  - 2⋅A_const⋅x₀ ₃ + A_const⋅x₁ ₀  + 2⋅A_const⋅x₁ ₀⋅x₁ ₁ + 2⋅A_const⋅

                                                               2              
x₁ ₀⋅x₁ ₂ + 2⋅A_const⋅x₁ ₀⋅x₁ ₃ - 2⋅A_const⋅x₁ ₀ + A_const⋅x₁ ₁  + 2⋅A_const⋅x

                                                              2               
₁ ₁⋅x₁ ₂ + 2⋅A_const⋅x₁ ₁⋅x₁ ₃ - 2⋅A_const⋅x₁ ₁ + A_const⋅x₁ ₂  + 2⋅A_const⋅x₁

                                       2      

# Functions to rotate coordinates with hubo variables

In [53]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [54]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [55]:
def generate_rotation_matrix(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = sp.expand(dx ** 2 + dy ** 2 + dz ** 2)
    l = sp.expand(sp.sqrt(l_sq))
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    for i in range(n_angles):
        c_theta += sp.expand(sp.cos(thetas[i]) * x[bond_no][i])
        s_theta += sp.expand(sp.sin(thetas[i]) * x[bond_no][i])
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = sp.expand((dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq)
    rotation_matrix[0, 1] = sp.expand((dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq)
    rotation_matrix[0, 2] = sp.expand((dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq)
    rotation_matrix[0, 3] = sp.expand(((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq)
    rotation_matrix[1, 0] = sp.expand((dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq)
    rotation_matrix[1, 1] = sp.expand((dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq)
    rotation_matrix[1, 2] = sp.expand((dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq)
    rotation_matrix[1, 3] = sp.expand(((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq)
    rotation_matrix[2, 0] = sp.expand((dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq)
    rotation_matrix[2, 1] = sp.expand((dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq)
    rotation_matrix[2, 2] = sp.expand((dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq)
    rotation_matrix[2, 3] = sp.expand(((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq)
    return sp.expand(rotation_matrix)

In [56]:
def rotate_all_coordinates():
    for i in coords_original.keys():
        if len(coords_rotation_dict[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_dict[i]:
                temp_rot_mat = generate_rotation_matrix(coords_original[torsional_bonds[bond_no][0]],
                                                        coords_original[torsional_bonds[bond_no][1]], bond_no)
                rot_mat = rot_mat * temp_rot_mat
            coords_dict[i] = rotate_coordinates(rot_mat, coords_original[i])

In [57]:
rotate_all_coordinates()

In [58]:
sp.pprint(coords_original)

{0: [1.0, -0.5, 0.0], 1: [0.0, 0.0, 0.0], 2: [0.0, 1.0, 0.0], 3: [1.0, 1.5, 0.
0], 4: [2.0, 1.0, 0.0]}


In [59]:
sp.pprint(coords_dict)

{0: [1.0, -0.5, 0.0], 1: [0.0, 0.0, 0.0], 2: [0.0, 1.0, 0.0], 3: [1.0⋅x₀ ₀ - x
₀ ₂, 1.5, -x₀ ₁ + 1.0⋅x₀ ₃], 4: [0.4⋅x₀ ₀⋅x₁ ₀ - 0.4⋅x₀ ₀⋅x₁ ₂ + 1.6⋅x₀ ₀ - 0.
894427190999916⋅x₀ ₁⋅x₁ ₁ + 0.894427190999916⋅x₀ ₁⋅x₁ ₃ - 0.4⋅x₀ ₂⋅x₁ ₀ + 0.4⋅
x₀ ₂⋅x₁ ₂ - 1.6⋅x₀ ₂ + 0.894427190999916⋅x₀ ₃⋅x₁ ₁ - 0.894427190999916⋅x₀ ₃⋅x₁
 ₃, -0.8⋅x₁ ₀ + 0.8⋅x₁ ₂ + 1.8, -0.894427190999916⋅x₀ ₀⋅x₁ ₁ + 0.8944271909999
16⋅x₀ ₀⋅x₁ ₃ - 0.4⋅x₀ ₁⋅x₁ ₀ + 0.4⋅x₀ ₁⋅x₁ ₂ - 1.6⋅x₀ ₁ + 0.894427190999916⋅x₀
 ₂⋅x₁ ₁ - 0.894427190999916⋅x₀ ₂⋅x₁ ₃ + 0.4⋅x₀ ₃⋅x₁ ₀ - 0.4⋅x₀ ₃⋅x₁ ₂ + 1.6⋅x₀
 ₃]}


# Functions to generate the optimization contraint in hubo

In [60]:
def distance_squared(first_coords, second_coords):
    dis_sq = sp.expand((second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 +(second_coords[2] - first_coords[2])**2)
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_dict[pair[0]], coords_dict[pair[1]])
    return distance_sq.expand()

In [61]:
distance_hubo = generate_distance_hubo()
sp.pprint(distance_hubo)

         2     2            2                      2                2     2   
0.48⋅x₀ ₀ ⋅x₁ ₀  - 0.96⋅x₀ ₀ ⋅x₁ ₀⋅x₁ ₂ + 3.84⋅x₀ ₀ ⋅x₁ ₀ + 2.4⋅x₀ ₀ ⋅x₁ ₁  - 

        2                      2     2            2                2     2    
4.8⋅x₀ ₀ ⋅x₁ ₁⋅x₁ ₃ + 0.48⋅x₀ ₀ ⋅x₁ ₂  - 3.84⋅x₀ ₀ ⋅x₁ ₂ + 2.4⋅x₀ ₀ ⋅x₁ ₃  + 9

        2                      2                                              
.68⋅x₀ ₀  - 0.96⋅x₀ ₀⋅x₀ ₂⋅x₁ ₀  + 1.92⋅x₀ ₀⋅x₀ ₂⋅x₁ ₀⋅x₁ ₂ - 7.68⋅x₀ ₀⋅x₀ ₂⋅x

                        2                                                2    
₁ ₀ - 4.8⋅x₀ ₀⋅x₀ ₂⋅x₁ ₁  + 9.6⋅x₀ ₀⋅x₀ ₂⋅x₁ ₁⋅x₁ ₃ - 0.96⋅x₀ ₀⋅x₀ ₂⋅x₁ ₂  + 7

                                       2                                      
.68⋅x₀ ₀⋅x₀ ₂⋅x₁ ₂ - 4.8⋅x₀ ₀⋅x₀ ₂⋅x₁ ₃  - 19.36⋅x₀ ₀⋅x₀ ₂ - 0.8⋅x₀ ₀⋅x₁ ₀ + 0

                                   2     2            2                      2
.8⋅x₀ ₀⋅x₁ ₂ - 5.2⋅x₀ ₀ + 0.48⋅x₀ ₁ ⋅x₁ ₀  - 0.96⋅x₀ ₁ ⋅x₁ ₀⋅x₁ ₂ + 3.84⋅x₀ ₁ 

                2     2           2           

# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [62]:
hubo_expr -= distance_hubo
hubo_expr_str = str(hubo_expr.expand())
f = open("full_hubo_expr.txt", "w")
print('full hubo expression written in file - full_hubo_expr.txt')
f.write(hubo_expr_str)
f.close()

full hubo expression written in file - full_hubo_expr.txt
